In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 15.9MB/s 
     |████████████████████████████████| 194kB 42.3MB/s 
     |████████████████████████████████| 1.2MB 31.9MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 47.9MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [2]:
from fastbook import *
from fastai.vision.all import *

path=Path("life")
cat=["animals","protista","plants","monera","fungi"]
key='81551da9516747d79840c932aafde189'
if not path.exists():
  path.mkdir(exist_ok=True)
  for id in cat:
    dest=(path/id)
    dest.mkdir(exist_ok=True)
    url=search_images_bing(key,id)
    res=url.attrgot("contentUrl")
    
    download_images(dest=dest,urls=res)
    
fns=get_image_files(path)
failed=verify_images(fns)
failed.map(Path.unlink)

data=DataBlock(blocks=(ImageBlock,CategoryBlock),item_tfms=Resize(128),get_y=parent_label,get_items=get_image_files)
dls=data.dataloaders(path)
learn=cnn_learner(dls,resnet34,metrics=error_rate)
learn.fine_tune(4)


 Download of http://miamitropicalplants.com/wp-content/uploads/2015/10/Variegated-Ming-Aralia.jpg has failed after 5 retries
 Fix the download manually:
$ mkdir -p life/plants
$ cd life/plants
$ wget -c http://miamitropicalplants.com/wp-content/uploads/2015/10/Variegated-Ming-Aralia.jpg
$ tar xf Variegated-Ming-Aralia.jpg
 And re-run your code once the download is successful



Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,error_rate,time
0,2.362059,0.754862,0.284615,01:21


epoch,train_loss,valid_loss,error_rate,time
0,0.861708,0.492674,0.169231,01:46
1,0.633728,0.494664,0.123077,01:48
2,0.471499,0.479395,0.123077,01:47
3,0.363390,0.441850,0.107692,01:47


In [5]:
learn.predict("life/animals/00000000.jpg")

('animals',
 tensor(0),
 tensor([1.0000e+00, 2.1767e-07, 4.3394e-08, 7.4179e-08, 5.8752e-09]))

In [4]:
learn.export()

In [32]:
from fastai.vision.widgets import *
from ipywidgets import *
lab=widgets.Label("Prediction will show here!")
out=widgets.Output()
model=load_learner("export.pkl")
btn1=widgets.FileUpload()
btn_run=widgets.Button(description="Run")
btn_r=widgets.Button(description="Remove")
def Classify(event):
  try:
    img=PILImage.create(btn1.data[0])
  except:
    lab.value="Sorry! File either not uploaded or unsupported"
    return
  with out:
    display(img.to_thumb(128))
  pred,te,prob=model.predict(img)
  lab.value=f'Predicition:{pred} ; Probabilty:{prob.max().item()}'
def remove(event):
  out.clear_output()
  btn1.data=[];
  lab.value='Prediction will show here!'
btn_run.on_click(Classify)
btn_r.on_click(remove)
VBox([widgets.Label("Upload the image of the organism"),out,btn1,btn_run,btn_r,lab])